# DIGI405 Lab Class: Sentiment Analysis

Make sure you use the Python 3.12 Kernel to run this notebook. 

This week’s class will investigate lexicon-based sentiment analysis with VADER (‘Valence Aware Dictionary for sEntiment Reasoning’). Vader is open source software, so you can inspect the code and modify it if you wish. In this week’s lab we will mainly refer to the lexicon.

Although VADER is more than 10 years old, it is still commonly used. You can learn lots about how language expresses sentiment by using VADER, understanding how it works, when it works and when it doesn't. 

The following cells imports libraries and creates a SentimentIntensityAnalyzer object.

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from IPython.display import display, HTML
from matplotlib.colors import LinearSegmentedColormap, to_hex

In [19]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt_tab to /home/gkf18/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [39]:
from textplumber.report import plot_confusion_matrix, plot_logistic_regression_features_from_pipeline
from textplumber.embeddings import Model2VecEmbedder
from textplumber.store import TextFeatureStore

# this is an in-development component being added into textplumber
from vader import VaderSentimentEstimator, VaderSentimentExtractor

In [21]:
custom_cmap = LinearSegmentedColormap.from_list("red_white_green", ["red", "white", "green"])

In [22]:
def norm_score(s):
    score_min = -1
    score_max = 1
    return (s - score_min) / (score_max - score_min) 

def highlight_row(row):
    normed = norm_score(row['compound'])
    color = to_hex(custom_cmap(normed))
    return [f'background-color: {color}; color: black'] * len(row)

In [23]:
pd.set_option('display.max_colwidth', None)

In [24]:
analyzer = SentimentIntensityAnalyzer()

## 1. Learn about VADER scores

In the cell below is a short phrase to show you the output of VADER. Get VADER's scores for the provided text and make sure you understand what each number tells us.

In [25]:
example = '''
This movie is terrible.
'''
vs = analyzer.polarity_scores(example)
print(str(vs))

{'neg': 0.508, 'neu': 0.492, 'pos': 0.0, 'compound': -0.4767}


Read the "About the Scoring" section of the Vader Github README, which explains the scores that are returned by Vader:  
https://github.com/cjhutto/vaderSentiment#about-the-scoring

### 1.1 Questions

1. What do the 'neu', 'pos', and 'neg' scores represent?  
2. What range of values of the Compound Score should be associated with a "neutral" classification?  


## 2. Score some text and understand Vader's lexicon and booster/negation rules

 

Here's another example - you can copy and paste this code into new code cells to test out different phrases.

In [26]:
example = '''
The movie was great.
'''
vs = analyzer.polarity_scores(example)
print(str(vs))

{'neg': 0.0, 'neu': 0.423, 'pos': 0.577, 'compound': 0.6249}


### 2.1 Activities

Try different text and make sure you understand the scores VADER returns. Copy the code above into new cells below for each example you come up with.

Create examples for the following conditions:   

1. A sentence that is obviously positive like "The movie is great"
2. A sentence that uses a "booster" e.g. "The movie is really terrible"
3. A sentence that uses negation e.g. "The movie is not great". 
4. Some sentences that attempts to fool Vader. Think about the discussion in class and challenges with sentiment classification in general, and specific challenges related to VADER's lexicon or rules.

### 2.2 Aide your understanding of VADER

Look at the lexicon and the booster/negation words on the VADER repository to get more insight into the scores.

* The VADER module code is here: https://github.com/nltk/nltk/blob/develop/nltk/sentiment/vader.py  
* Negations and booster words are on lines 48-181.  
* The Vader lexicon is available here: https://github.com/cjhutto/vaderSentiment/blob/master/vaderSentiment/vader_lexicon.txt  Note: you can search the lexicon in your browser or you can download it and inspect it in a text editor.  
* Make sure you are clear what the values in the VADER lexicon actually mean.  

Here are some examples for your reference: 

    hope 	1.9 0.53852 [3, 2, 2, 1, 2, 2, 1, 2, 2, 2]
    hopeless -2.0 1.78885 [-3, -3, -3, -3, 3, -1, -3, -3, -2, -2]

* The VADER paper itself is helpful also: https://ojs.aaai.org/index.php/ICWSM/article/view/14550

## 3. Score longer texts

Below we load the movie reviews dataset we used in a previous lab. 

You can browse the dataset here: https://huggingface.co/datasets/polsci/sentiment-polarity-dataset-v2.0
or download the texts here: https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/movie_reviews.zip 

In [27]:
dataset = load_dataset('polsci/sentiment-polarity-dataset-v2.0')
label_column = 'label'
X = dataset['train']['text']
y = dataset['train'][label_column]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
target_names = dataset['train'].features[label_column].names
target_classes = list(range(len(target_names)))

### 3.1 Activity:

Run the cells below to preview a review and get VADER's scores.

Try some different reviews from the dataset and see what scores Vader comes up with. Are the scores correct against the actual label?

In [28]:
review_id = 904
try:
    review = X_train[review_id]
    print(f"Label: {target_names[y_train[review_id]]}")
    print()
    print(review)
except IndexError:
    print(f"Review ID {review_id} is out of range for the training set.")

Label: pos

no film in recent has left me with such conflicted feelings as neil jordan's harrowing , humorous , horrifying adaptation of patrick mccabe's novel about young lad francie brady's ( eamonn owens ) descent into madness in 1960s ireland . 
on one hand , it was difficult for me to become invested in francie's story because he is such an unsavory character , unjustifyably venting his rage at his nosy but otherwise harmless neighbor mrs . nugent ( fiona shaw ) . 
on another hand , i found it difficult to laugh at some of francie's darkly comic shenanigans because he obviously is such a sick , needy child , having been raised by a drunken father ( stephen rea ) and a suicidal mother ( aisling o'sullivan ) . 
on yet another hand , i also found it difficult to completely sympathize with francie during his more emotional scenes because some of his , for lack of a better word , " bad " deeds are so incredibly shocking in their brutality and the malicious glee in which he performs the

In [29]:
vs = analyzer.polarity_scores(review)
print(str(vs))

{'neg': 0.148, 'neu': 0.706, 'pos': 0.146, 'compound': 0.0831}


### 3.2. Evaluating VADER's performance on long texts

This cells below compare VADER scoring and a model based on classification of embeddings. 

First, here is an evaluation of VADER.

Note: The labels in the dataset are either positive or negative (i.e. no neutral). Here compound scores greater than or equal to 0 are considered positive, and scores less than 0 are considered negative.

In [30]:
pipeline = Pipeline([
        ('classifier', VaderSentimentEstimator(output = 'labels', neutral_threshold = 0, label_mapping = {'positive': 1, 'negative': 0})),
], verbose=True)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred, labels = target_classes, target_names = target_names, digits=3))
plot_confusion_matrix(y_test, y_pred, target_classes, target_names)

[Pipeline] ........ (step 1 of 1) Processing classifier, total=   0.0s
              precision    recall  f1-score   support

         neg      0.727     0.465     0.567       200
         pos      0.607     0.825     0.699       200

    accuracy                          0.645       400
   macro avg      0.667     0.645     0.633       400
weighted avg      0.667     0.645     0.633       400



Second, here is a model based on a Logistic Regression classifier and Model2Vec embeddings on the same dataset. 

In [31]:
feature_store = TextFeatureStore('sentiment-lab-movie_reviews.sqlite')
pipeline = Pipeline([
        ('features', Model2VecEmbedder(feature_store = feature_store)),
        ('classifier', LogisticRegression(max_iter = 5000, random_state=55))
], verbose=True)
display(pipeline)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, labels = target_classes, target_names = target_names, digits=3))
plot_confusion_matrix(y_test, y_pred, target_classes, target_names)

Pipeline(steps=[('features',
                 Model2VecEmbedder(feature_store=<textplumber.store.TextFeatureStore object at 0x7f15d17c72f0>)),
                ('classifier',
                 LogisticRegression(max_iter=5000, random_state=55))],
         verbose=True)

[Pipeline] .......... (step 1 of 2) Processing features, total=   0.1s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   0.0s
              precision    recall  f1-score   support

         neg      0.737     0.770     0.753       200
         pos      0.759     0.725     0.742       200

    accuracy                          0.748       400
   macro avg      0.748     0.748     0.747       400
weighted avg      0.748     0.748     0.747       400



The labels based on VADER scores are accurate more often than not, but accuracy is not great on these long texts (around 65% on this dataset) when compared to a basic classifier (around 75%). Observe that on these long reviews, VADER has a tendency to label reviews as positive more than negative. 

VADER works better on short texts. The original VADER paper indicates it worked best on social media texts.

Despite these limitations, we can use VADER to explore some of the problems deriving overall sentiment scores using a lexicon-based approach and some of the challenges of measuring sentiment more generally.

## 4. Examining sentiment scores by sentence

Let’s look at an example review to think about the different frames of reference to which sentiments might be connected. The example we will use is a review of Neil Jordan’s film The Butcher Boy filename cv079_11933.txt. 

A descriptive statement describes the content of the film. Eg sentence 3: Francie is a “sick, needy child” - this tells us about what happens in the film.

An analytic statement analyses the content of the film. 

Eg sentence 3: “I found it difficult to laugh at some of Francie’s darkly comic shenanigans” - here the reviewer is analysing the effects of the film.

It’s not a perfect distinction, but we can observe that negative content in the film doesn’t necessarily imply a negative review of the film. Both types of statements can include evaluative language and include indications of the reviewer's point of view about the movie, but lexicon-based sentiment analysis will have difficulty if a review has a lot of “negative” content, but is nonetheless given a positive review.



### 4.1. Activity
Run the following cells to split the text into sentences and output scores for each sentence.

In [32]:
review_id = 904 # change the review ID to examine another review
try:
    review = X_train[review_id]
    # this splits the review using NLTK's sentence tokenizer and removes empty sentences or sentences with only common punctuation
    sentences = sent_tokenize(review)
    sentences = [s for s in sentences if s.strip() and s.strip() not in ['.', ',', '!', '?']]
    print(f"Label: {target_names[y_train[review_id]]}")
    print()
    display(sentences)
except IndexError:
    print(f"Review ID {review_id} is out of range for the training set.")

Label: pos



["no film in recent has left me with such conflicted feelings as neil jordan's harrowing , humorous , horrifying adaptation of patrick mccabe's novel about young lad francie brady's ( eamonn owens ) descent into madness in 1960s ireland .",
 "on one hand , it was difficult for me to become invested in francie's story because he is such an unsavory character , unjustifyably venting his rage at his nosy but otherwise harmless neighbor mrs .",
 'nugent ( fiona shaw ) .',
 "on another hand , i found it difficult to laugh at some of francie's darkly comic shenanigans because he obviously is such a sick , needy child , having been raised by a drunken father ( stephen rea ) and a suicidal mother ( aisling o'sullivan ) .",
 'on yet another hand , i also found it difficult to completely sympathize with francie during his more emotional scenes because some of his , for lack of a better word , " bad " deeds are so incredibly shocking in their brutality and the malicious glee in which he performs 

In [33]:
data = []
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    data.append([sentence, vs['neg'], vs['neu'], vs['pos'], vs['compound']])

df = pd.DataFrame(data, columns=['sentence','neg','neu','pos','compound'])

print(f"Label: {target_names[y_train[review_id]]}")
print()
display(df.style.apply(highlight_row, axis=1))

Label: pos



,sentence,neg,neu,pos,compound
0,"no film in recent has left me with such conflicted feelings as neil jordan's harrowing , humorous , horrifying adaptation of patrick mccabe's novel about young lad francie brady's ( eamonn owens ) descent into madness in 1960s ireland .",0.181000,0.719000,0.101000,-0.599400
1,"on one hand , it was difficult for me to become invested in francie's story because he is such an unsavory character , unjustifyably venting his rage at his nosy but otherwise harmless neighbor mrs .",0.144000,0.745000,0.111000,-0.102700
2,nugent ( fiona shaw ) .,0.000000,1.000000,0.000000,0.000000
3,"on another hand , i found it difficult to laugh at some of francie's darkly comic shenanigans because he obviously is such a sick , needy child , having been raised by a drunken father ( stephen rea ) and a suicidal mother ( aisling o'sullivan ) .",0.207000,0.683000,0.111000,-0.709600
4,"on yet another hand , i also found it difficult to completely sympathize with francie during his more emotional scenes because some of his , for lack of a better word , "" bad "" deeds are so incredibly shocking in their brutality and the malicious glee in which he performs them .",0.217000,0.613000,0.170000,-0.523300
5,"however , the butcher boy's power is undeniable , and the film as a whole is unforgettable--perhaps because it is so disturbing .",0.162000,0.838000,0.000000,-0.641800
6,"what makes it so unsettling is the francie's overall wink-wink yet matter-of-fact attitude about everything , expressed in a cheeky voiceover narration delivered by the adult francie ( rea again ) .",0.000000,1.000000,0.000000,0.000000
7,"think heavenly creatures played largely for laughs , and you'll sort of understand .",0.000000,0.534000,0.466000,0.862500
8,"anchoring the whole film is the astonishing debut performance of owens ; love francie or hate him , you cannot take your eyes off of owens .",0.112000,0.760000,0.128000,0.128000
9,"the butcher boy truly is a twisted , unusual film that is bound to make just about anyone uncomfortable .",0.111000,0.766000,0.123000,0.077200


### 4.2. Questions

1. Look closely at each sentence and work out which ones relate to the reviewer's evaluation of the movie. Is Vader doing a good job of scoring these sentences?  
2. Try this with another review. Change the ID number in the cell above to load another review. Look carefully at the positively and negatively evaluated sentences using the compound score. From this analysis, what challenges do you see in correctly assigning overall sentiment scores to movie reviews?

## 5. Examining the structure of reviews

In class we talked about the argumentative structure of reviews, what reviewers are doing when they write a review and who a review is for. When evaluating a film, reviewers rarely just say "Loved it" or "Hated it", that is what the number rating is for. Reviewers tend to craft an argument that justifies their rating, using the descriptive and analytical statements discussed above. Reviewers also tend to follow conventions of other reviews they've read and anticipate that people read reviews to find movies to watch. For a reviewer to be viewed as credible and their review to be useful to its potential audience, reviewers will often point out positive and negative features of a film, while expressing their evaluation. This weighing up of good and bad may help readers understand if a film is suitable for them or not. In a positive review, we can expect some discussion of negative features, and in a negative review we can expect some discussion of positive features. 

This point is not just about reviews, this is a general point about the structure of opinion-giving. Part of giving your view is anticipating the views of others.  

### 5.1 Activity

Below is an example of a review that discusses positive and negative features of a film and discusses who the film might be suitable for. Change the ID number and examine how other review authors are orienting to their audience and structuring their evaluation.

In [34]:
review_id = 214 # change the review ID to examine another review
try:
    review = X_train[review_id]
    # this splits the review using NLTK's sentence tokenizer and removes empty sentences or sentences with only common punctuation
    sentences = sent_tokenize(review)
    sentences = [s for s in sentences if s.strip() and s.strip() not in ['.', ',', '!', '?']]

    data = []
    for sentence in sentences:
        vs = analyzer.polarity_scores(sentence)
        data.append([sentence, vs['neg'], vs['neu'], vs['pos'], vs['compound']])

    df = pd.DataFrame(data, columns=['sentence','neg','neu','pos','compound'])

    print(f"Label: {target_names[y_train[review_id]]}")
    print()
    display(df.style.apply(highlight_row, axis=1))
except IndexError:
    print(f"Review ID {review_id} is out of range for the training set.")


Label: pos



,sentence,neg,neu,pos,compound
0,"when casting the key part of the voice of moses , most people's first candidate would probably not be the controversial actor val kilmer ( the saint and the island of dr .",0.000000,0.953000,0.047000,0.151100
1,"moreau ) , but he was dreamworks's choice in their first traditionally animated movie , the prince of egypt .",0.000000,1.000000,0.000000,0.000000
2,"the selection of kilmer proved to be a wise one , but the biggest risk dreamworks took wasn't in the casting department .",0.103000,0.817000,0.080000,-0.153100
3,"the radical departure was that they decided to use the medium of animation to make a biblical epic , a la cecil b .",0.000000,1.000000,0.000000,0.000000
4,"demille , to move our hearts and minds rather than to tickle our funny bones .",0.000000,0.660000,0.340000,0.802000
5,in the press kit producer penney cox says that they wanted to make a film for adults based on sophisticated themes but one that did not exclude children .,0.000000,0.863000,0.137000,0.510400
6,"animated movies for adults that are appropriate for kids and that are not comedies , now that is something fresh .",0.096000,0.807000,0.098000,0.010800
7,let me admit up-front that i was quite skeptical as to whether dreamworks could pull it off .,0.127000,0.785000,0.088000,-0.200600
8,dreamworks's antz seemed to be a movie not quite appropriate for any age group .,0.000000,1.000000,0.000000,0.000000
9,"the prince of egypt may be harder to market than antz , but it is much more satisfying .",0.000000,0.802000,0.198000,0.664000


### 5.2. Looking at structure across the corpus

The following visualisation shows some interesting patterns from exploratory analysis of the review corpus. The visualisation clusters reviews by the structure of sentiment scores. Take a look at the visualisation now. There are some notes at the bottom of the image to help you interpret it.

In [35]:
VaderSentimentEstimator(output = 'labels', neutral_threshold = 0, label_mapping = {'positive': 1, 'negative': 0}).plot_sentiment_structure(X_train, y_train, target_classes = target_classes, target_names = target_names)

### 5.3. Questions about the visualisation

1. What are some differences you notice between positive and negative reviews?  
2. Are there clusters you would expect to be misclassified by VADER across the whole document?

## 5. What happens if we classify based on VADER scores across documents?

The following cell trains a model based on multiple sentiment features for each text. The 'profile' value extracts a document sentiment profile. This includes the overall VADER compound score and positive/negative/neutral proportions, but also sentiment scores across the structure of the texts. Compound scores are extracted for the first three sentences, the last three sentences and four random sentences from the middle of the text. Short texts are handled by padding the extracted features with zeros. 

This takes a while!

In [36]:
pipeline = Pipeline([
        ('features', VaderSentimentExtractor(output = 'profile')),
        ('classifier', LogisticRegression(max_iter = 5000, random_state=55))
], verbose=True)
display(pipeline)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, labels = target_classes, target_names = target_names, digits=3))
plot_confusion_matrix(y_test, y_pred, target_classes, target_names)

Pipeline(steps=[('features', VaderSentimentExtractor(output='profile')),
                ('classifier',
                 LogisticRegression(max_iter=5000, random_state=55))],
         verbose=True)

[Pipeline] .......... (step 1 of 2) Processing features, total=  35.5s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   0.0s
              precision    recall  f1-score   support

         neg      0.714     0.675     0.694       200
         pos      0.692     0.730     0.710       200

    accuracy                          0.703       400
   macro avg      0.703     0.703     0.702       400
weighted avg      0.703     0.703     0.702       400



Although not reaching the performance of the model based on emebddings, this model outperforms the overall VADER scoring of long texts. These sentiment features can be combined with other features to improve performance further.

The plot of discriminative features is interesting, as it shows the importance of sentence scores in the conclusion of reviews to improve classification performance. 

In [37]:
plot_logistic_regression_features_from_pipeline(pipeline, target_classes, target_names, top_n=20, classifier_step_name = 'classifier', features_step_name = 'features')

,Feature,Log Odds (Logit),Odds Ratio
3,doc_positive,2.249902,9.486804
1,doc_negative,-1.291268,0.274922
2,doc_neutral,-0.918968,0.398931
9,conclusion_sentence_2,0.857769,2.357895
7,conclusion_sentence_0,0.510731,1.666509
8,conclusion_sentence_1,0.415764,1.515529
0,doc_compound,0.404981,1.499274
13,body_sentence_sample_3,0.312292,1.366553
10,body_sentence_sample_0,0.267535,1.306740
6,introduction_sentence_2,0.246199,1.279154


## 5. Concluding Activities and Questions

1. **ACTIVITY:** In class this week we discussed how sentiment analysis might not be an appropriate technique for analysing some kinds of texts. For example, some texts are not primarily about presenting a point of view or evaluation (e.g. journalistic texts, scientific writing) and authors/speakers don't always present their evaluations in a straightforward way (e.g. some political texts). Take some time to explore some different kinds of texts (e.g. editorials, fiction, tweets, news articles, political speeches, texts from the corpus you built for the Corpus Building Project). Vader will tend to perform better with short texts, so make sure you try texts of different lengths.  
**QUESTION:** How does Vader perform on different kinds of texts? What kinds of texts are challenging for a lexicon-based approach to sentiment analysis? What kinds of texts are not appropriate for sentiment analysis?

In [38]:
example = '''
Put your text samples here.
'''
vs = analyzer.polarity_scores(example)
print(str(vs))

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


2. **ACTIVITY:** Under the Readings section on this week's AKO|LEARN page is a link to a Hugging Face Spaces that allows you to test a pre-trained models for Sentiment Analysis. We have also linked to the relevant model the web app is using. Try out some of the sentences from the movie review example above. Try other texts you have tested in the lab today.  
**QUESTION:** How do these models perform compared with Vader? What are some of the advantages and disadvantages of using pre-trained machine learning models?

**Discuss what you have found with your neighbour. If you have time at the end of the lab you can work on your assignment.**